In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

In [2]:
da = pd.read_parquet('data/de_train.parquet') #  data_de_train

In [3]:
columns = da.columns
Y = da.loc[:, columns[5:]]

### get the mean and the variance of Y

In [137]:
column_means = Y.mean()

# Calculate the standard deviation for each column
column_std = Y.std()


In [4]:
columns[0:7]

Index(['cell_type', 'sm_name', 'sm_lincs_id', 'SMILES', 'control', 'A1BG',
       'A1BG-AS1'],
      dtype='object')

In [5]:
X = da.loc[:, columns[0:5]]
select_col = ['cell_type', 'sm_name', 'control']
X = X[select_col]

(614, 3)

In [ ]:
############# X 是自变量，Y是因变量，对X进行编码

In [6]:
# X.describe

In [7]:
X_encoded = pd.get_dummies(X, columns=X.columns)

In [29]:
X_encoded.columns[0:8]

Index(['cell_type_B cells', 'cell_type_Myeloid cells', 'cell_type_NK cells',
       'cell_type_T cells CD4+', 'cell_type_T cells CD8+',
       'cell_type_T regulatory cells',
       'sm_name_5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine',
       'sm_name_ABT-199 (GDC-0199)'],
      dtype='object')

In [8]:
np.shape(X_encoded)

(614, 154)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=13)

In [10]:
np.shape(y_train)

(491, 18211)

In [11]:
pca_y = PCA(n_components=10)  # 例如，降维到10个主成分
y_train_pca = pca_y.fit_transform(y_train)
y_test_pca = pca_y.transform(y_test)

In [25]:
np.shape(y_train_pca)

(491, 10)

#### 计算现有主成分的累计贡献率

In [28]:
# Get the explained variance ratio from the PCA object
explained_variance_ratio = pca_y.explained_variance_ratio_

cumulative_explained_variance = np.cumsum(explained_variance_ratio)

print(cumulative_explained_variance[len(cumulative_explained_variance)-1])

0.8874122525511449


##### 累计贡献率达到88%，暂时视为满意

In [ ]:
#### 对10个主成分分别建模并进行预测后再整合输出，得到pre后再进行反变换

In [30]:
assert X_train.shape[0] == y_train_pca.shape[0]

In [31]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

In [32]:
# Initialize the SVR model
svr = SVR()

In [33]:
# Using MultiOutputRegressor to handle multiple target values (10 principal components)
model = MultiOutputRegressor(svr)

In [34]:
# Train the model
model.fit(X_train, y_train_pca)

MultiOutputRegressor(estimator=SVR())

In [35]:
# Predict on new data (X_test)
y_test_pca_pred = model.predict(X_test)

# Convert back to the original target space (if necessary)
y_test_pred = pca_y.inverse_transform(y_test_pca_pred)

In [36]:
y_test_pred.shape

(123, 18211)

In [37]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_test_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared: {r2}")


Mean Absolute Error (MAE): 0.822861088384184
Mean Squared Error (MSE): 4.759593667011635
Root Mean Squared Error (RMSE): 2.1816492997298247
R-squared: -0.020591644902474806


In [55]:
submission = pd.read_csv('data/sample_submission.csv') #  提交

In [42]:
id_pair = pd.read_csv('data/id_map.csv') #  每个id对应的药物与细胞类型
id_col = ["cell_type", "sm_name"]

In [45]:
new_id = id_pair.loc[:,id_col]
new_id = pd.get_dummies(new_id, columns=new_id.columns)

In [50]:
# Columns that are in X but not in new
missing_cols = set(X_test.columns) - set(new_id.columns)

# Add these columns to new and set their values to zero
for col in missing_cols:
    new_id[col] = 0

# If you want to reorder the columns of new to match the order of X's columns
new_id = new_id[X_test.columns]

In [52]:
sub_0 = model.predict(new_id)
sub_1 = pca_y.inverse_transform(sub_0)


In [131]:
sub_1.columns

RangeIndex(start=0, stop=18211, step=1)

In [134]:

# Convert to DataFrame if it isn't already
if isinstance(sub_1, np.ndarray):
    sub_1 = pd.DataFrame(sub_1)
# Ensure column_means and column_std are Series with the right index
column_means = pd.Series(column_means)
column_std = pd.Series(column_std, index = list(range(18211)))

# Perform the operation
# normalized_sub_1 = (sub_1 - column_means) / column_std


In [141]:
column_means = pd.Series(column_means)
column_means.index = range(18211)

column_std = pd.Series(column_std)
column_std.index = range(18211)
column_std[0:10]

0    1.634697
1    1.188660
2    2.275410
3    2.184095
4    3.189206
5    3.826391
6    0.703410
7    1.259422
8    0.952243
9    1.237806
dtype: float64

In [155]:
normalized_sub_1 = (sub_1 + column_means)*column_std

In [156]:
normalized_sub_1 

0         1         2         3         4         5         6      \
0    0.745539  0.349828  0.449714  0.867938  3.103076  4.359756 -0.060373   
1    0.733751  0.342792  0.381572  0.783382  3.040424  4.389624 -0.060232   
2    0.742561  0.348634  0.408490  0.822312  3.044190  4.416726 -0.059580   
3    0.741872  0.345592  0.407869  0.814366  3.118400  4.338453 -0.059338   
4    0.742841  0.348087  0.429872  0.802874  3.108599  4.373121 -0.061675   
..        ...       ...       ...       ...       ...       ...       ...   
250  0.726093  0.352052  0.203153  0.654267  2.879877  4.276848 -0.057398   
251  0.733155  0.353695  0.237139  0.678896  2.918041  4.329521 -0.058094   
252  0.733854  0.353678  0.219727  0.673751  2.938228  4.290845 -0.057243   
253  0.735585  0.358061  0.229641  0.671925  2.927259  4.310157 -0.058170   
254  0.729616  0.355516  0.225755  0.663906  2.860124  4.315669 -0.057547   

        7         8         9      ...     18201     18202     18203  \
0    0.608132 -0.114288  0.217546  ... -0.025972  0.134129 -0.094903   
1    0.597667 -0.104403  0.193849  ... -0.008764  0.129080 -0.104940   
2    0.603005 -0.106690  0.198735  ... -0.012177  0.132458 -0.098921   
3    0.605781 -0.102153  0.195373  ... -0.004651  0.138679 -0.100165   
4    0.607959 -0.107283  0.213938  ... -0.015357  0.134560 -0.097909   
..        ...       ...       ...  ...       ...       ...       ...   
250  0.601554 -0.105793  0.149112  ...  0.001695  0.117715 -0.119239   
251  0.603367 -0.112481  0.170807  ... -0.010268  0.117916 -0.113805   
252  0.606015 -0.112227  0.156221  ... -0.006403  0.117971 -0.116981   
253  0.607518 -0.110991  0.174219  ... -0.011882  0.117447 -0.115155   
254  0.605835 -0.110706  0.152391  ... -0.006124  0.118695 -0.113475   

        18204     18205     18206     18207     18208     18209     18210  
0    0.433889  1.163311  0.725929  0.370895  0.216266 -0.240090 -0.114608  
1    0.429903  1.128430  0.705486  0.364081  0.212624 -0.237992 -0.117714  
2    0.434713  1.143643  0.716275  0.367331  0.214288 -0.237896 -0.117146  
3    0.433402  1.152406  0.719061  0.369074  0.213536 -0.241764 -0.115044  
4    0.434232  1.159012  0.721830  0.369836  0.217073 -0.245855 -0.113810  
..        ...       ...       ...       ...       ...       ...       ...  
250  0.394806  1.144952  0.721342  0.352745  0.199589 -0.253589 -0.122738  
251  0.399600  1.155394  0.724983  0.355206  0.203741 -0.254866 -0.121555  
252  0.397092  1.162825  0.730299  0.354783  0.200019 -0.255781 -0.121275  
253  0.396366  1.160511  0.729332  0.354225  0.203480 -0.255820 -0.121566  
254  0.400208  1.153957  0.728974  0.355704  0.201138 -0.254890 -0.122144  

[255 rows x 18211 columns]

#### 缺失值计算

In [157]:
# Convert the numpy array to DataFrame
# sub_1_df = pd.DataFrame(normalized_sub_1, columns=submission.columns[1:])
normalized_sub_1.columns = submission.columns[1:]
# Use the 'ID' column from submission and concatenate it with sub_1_df
result = pd.concat([submission['id'], normalized_sub_1], axis=1)

In [158]:
result

id      A1BG  A1BG-AS1       A2M   A2M-AS1     A2MP1    A4GALT  \
0      0  0.745539  0.349828  0.449714  0.867938  3.103076  4.359756   
1      1  0.733751  0.342792  0.381572  0.783382  3.040424  4.389624   
2      2  0.742561  0.348634  0.408490  0.822312  3.044190  4.416726   
3      3  0.741872  0.345592  0.407869  0.814366  3.118400  4.338453   
4      4  0.742841  0.348087  0.429872  0.802874  3.108599  4.373121   
..   ...       ...       ...       ...       ...       ...       ...   
250  250  0.726093  0.352052  0.203153  0.654267  2.879877  4.276848   
251  251  0.733155  0.353695  0.237139  0.678896  2.918041  4.329521   
252  252  0.733854  0.353678  0.219727  0.673751  2.938228  4.290845   
253  253  0.735585  0.358061  0.229641  0.671925  2.927259  4.310157   
254  254  0.729616  0.355516  0.225755  0.663906  2.860124  4.315669   

         AAAS      AACS     AAGAB  ...      ZUP1      ZW10    ZWILCH  \
0   -0.060373  0.608132 -0.114288  ... -0.025972  0.134129 -0.094903   
1   -0.060232  0.597667 -0.104403  ... -0.008764  0.129080 -0.104940   
2   -0.059580  0.603005 -0.106690  ... -0.012177  0.132458 -0.098921   
3   -0.059338  0.605781 -0.102153  ... -0.004651  0.138679 -0.100165   
4   -0.061675  0.607959 -0.107283  ... -0.015357  0.134560 -0.097909   
..        ...       ...       ...  ...       ...       ...       ...   
250 -0.057398  0.601554 -0.105793  ...  0.001695  0.117715 -0.119239   
251 -0.058094  0.603367 -0.112481  ... -0.010268  0.117916 -0.113805   
252 -0.057243  0.606015 -0.112227  ... -0.006403  0.117971 -0.116981   
253 -0.058170  0.607518 -0.110991  ... -0.011882  0.117447 -0.115155   
254 -0.057547  0.605835 -0.110706  ... -0.006124  0.118695 -0.113475   

        ZWINT      ZXDA      ZXDB      ZXDC    ZYG11B       ZYX     ZZEF1  
0    0.433889  1.163311  0.725929  0.370895  0.216266 -0.240090 -0.114608  
1    0.429903  1.128430  0.705486  0.364081  0.212624 -0.237992 -0.117714  
2    0.434713  1.143643  0.716275  0.367331  0.214288 -0.237896 -0.117146  
3    0.433402  1.152406  0.719061  0.369074  0.213536 -0.241764 -0.115044  
4    0.434232  1.159012  0.721830  0.369836  0.217073 -0.245855 -0.113810  
..        ...       ...       ...       ...       ...       ...       ...  
250  0.394806  1.144952  0.721342  0.352745  0.199589 -0.253589 -0.122738  
251  0.399600  1.155394  0.724983  0.355206  0.203741 -0.254866 -0.121555  
252  0.397092  1.162825  0.730299  0.354783  0.200019 -0.255781 -0.121275  
253  0.396366  1.160511  0.729332  0.354225  0.203480 -0.255820 -0.121566  
254  0.400208  1.153957  0.728974  0.355704  0.201138 -0.254890 -0.122144  

[255 rows x 18212 columns]

In [162]:
result.to_csv('pre_1.csv', index=False)